### 序列相关
　　序列最重要的特征就是可包含多个元素， 因此和序列有关的特殊方法有如下几个：<br>

```python
__len__(self)：该方法的返回值决定序列中元素的个数。<br>
__getitem__(self, key)：该方法获取指定索引对应的元素。该方法的 key 应该是整数值或 slice 对象，否则该方法会引发 KeyError 异常。<br>
__contains__(self, item)：该方法判断序列是否包含指定元素。<br>
__setitem__(self, key, value)：该方法设置指定索引对应的元素。该方法的 key 应该是整数值或 slice 对象，否则该方法会引发 KeyError 异常。<br>
__delitem__(self, key)：该方法删除指定索引对应的元素。<br>
```

　　如果程序要实现不可变序列（程序只能获取序列中的元素，不能修改），只要实现上面前 3 个方法就行；如果程序要实现可变序列（程序既能获取序列中的元素，也可修改），则需要实现上面 5 个方法。<br>
  
## 二、属性相关魔法方法
### 2.1、\_\_getattr\_\_  & hasattar()
#### 2.1.1、hasattar()
　　判断某个属性是否存在于某个类的属性集合中（包括函数，函数也是属性）,如果不存在,则返回false。
#### 2.1.2、\_\_getattribute\_\_ 


　　以如下代码为例：创建一个A对象，并给所有的A的实例分配了一个属性value,每个实例都有此属性，只是不共享（注意，这不是类对象的属性，类对象的属性对类的实例来说，是全局变量）。那么，我们可以利用hasattar()判断一个属性是不是在一个对象属性的集合中。

```python
class A(object):
    name = "小明"
    def __init__(self, value):
        self.value = value

    def __getattr__(self, item):
        print("找不到此属性")
        return super().__getattribute__(item)

# test 1
a = A(10)
print(hasattr(a,"value"))
# 结果为:
# True

# test 2
print(hasattr(a,"value1"))
# 结果为:
# 找不到此属性
# False

# test 3
print(hasattr(a,"value"))
print(hasattr(A,"value"))
print(hasattr(A,"name"))
# 结果为：
# True
# False
# True
```
　　由test 1 判断: value确实是属于A的实例的属性，结果为True。那么我们把value改成value1呢？  
　　由test 2 判断：value1并不是A实例的属性。并且从代码中可以看出，调用hasattr()时，会自动调用\_\_getattr\_\_（）去访问A实例的属性，如果类实例没有这个属性，就会返回False。  
　　由test 3 判断：事实上我们不仅可以在hasattr()函数中的第一个参数放入类实例，也可以放类对象，因为类也是一个对象，也拥有属性。但是我们不能把hasattr()函数的第二个参数放类实例的属性，因为实例的属性只属于类实例，而不属于类对象，类属性对类实例来说，是全局变量。从test 3结果可以得知，第二个结果为False，说明前面说的“实例的属性只属于类实例，而不属于类对象，类属性对类实例来说，是全局变量”这句话是对的。有人可能奇怪，为什么“print(“找不到此属性”)”对应的这个函数为什么不执行，找不到属性的话，不是会执行\_\_getattr\_\_()函数吗？当然会执行，只不过执行的是A对象上层类的\_\_getattr\_\_()函数。    

#### 2.1.3、getattr()
　　获取某个对象的属性值，如果获取不到，则执行\_\_getattr\_\_函数抛出异常。

```python
class A(object):
    name = "小明"
    def __init__(self, value):
        self.value = value
        self.key = "key"

    def __getattr__(self, item):
        print("找不到此属性")
        return super().__getattribute__(item)

    def test(self):
        print("哈哈")
        return "正在测试中"

a = A(10)

# test 4
print(getattr(a,"key"))
#结果为：10

# test 5   稍微该一行代码
print(getattr(a,"key1"))　　# key1属性肯定不存在于类实例中的，结果会报错
# 结果：找不到此属性
Traceback (most recent call last):
  File "C:/Users/Administrator/Desktop/pm2/pm2/app1/tests.py", line 17, in <module>
    print(getattr(a,"key1"))
  File "C:/Users/Administrator/Desktop/pm2/pm2/app1/tests.py", line 9, in __getattr__
    return super().__getattribute__(item)
AttributeError: 'A' object has no attribute 'key1'
```

　　还可以对print(getattr(a,"key1"))这一行代码进行改造，加上一个默认值，print(getattr(a,"key1","我是默认值")),如果实例中没有"key1"这个属性，getattr()就会返回这个默认值，防止程序报错。  
　　另一种情况，假如那我们把第二个参数传递为方法属性呢？同样可以获取方法属性的值，即方法地址。如果想执行的话，加个()括号就可以调用了。  

```python
# test 6   接上面的测试  传入方法属性
print(getattr(a,"test1","我是默认值"))
# 结果为：  
# <bound method A.test of <__main__.A object at 0x00000280976408C8>

# test 7  加上括号带用呢
print(getattr(a,"test1","我是默认值")())
# 结果为：
# 哈哈
# 正在测试
```

### 2.2、\_\_setattr\_\_ & setattr()
　　属性赋值方法
```python
class A(object):
    name = "小明"
    def __init__(self, value):
        self.value = value
        self.key = "key"

    def __setattr__(self, key, value):
        self.__dict__[key] = value
        print("__setattr__被调用")


    def test(self):
        print("哈哈")
        return "正在测试中"

a = A(10)

# test 1
setattr(a,"key","我是修改后的key")
print(a.key)
# 结果：
# __setattr__被调用
# 我是修改后的key

# test 2
setattr(a,"new_key","我是新建立的key")
print(a.new_key)
# 结果为：
# __setattr__被调用
# __setattr__被调用
# __setattr__被调用
# 我是新建立的key

```

　　test 1: 当试图对象的item特性赋值的时，\_\_setattr\_\_() 将会被调用。\_\_setattr\_\_依赖setattr（）而存在。(其实也可以理解为，一个类实例就是一个字典，属性为”key“,值为"value",其实就是修改这个字典的key所对应的值)。  
　　给对象的属性赋值，若属性不存在，则先创建再赋值。 
　　test 2: 前两个"\_\_setattr\_\_被调用"是因为\_\_init\_\_函数里有两次赋值行为，会默认调用\_\_setattr\_\_方法，最后一个才是因为 "setattr(a,"new_key","我是新建立的key")"这句代码。  
　　通过 setattr() 进行属性赋值和通过 对象.属性=xxx 进行赋值或者修改并无区别，都会调用\_\_setattr\_\_方法，可以结合字典的用法，无非就是增删查改，事实上，和字典用法并无实质区别（也可以结合《流畅的python》这本书，看一下，如何将字典模拟成类进行，然后进行类似于类的操作）：
```python
class A(object):
    # name = "小明"
    def __init__(self, value):
        self.value = value
        self.key = "key"


    def __setattr__(self, key, value):
        self.__dict__[key] = value
        print("__setattr__被调用")


    def test(self):
        print("哈哈")
        return "正在测试中"

a = A(10)
setattr(a,"new_key","我是新建立的key")
print(a.new_key)
a.new_key=2

# 结果为：
# __setattr__被调用
# 我是新建立的key
# __setattr__被调用
```
